# Resume Recomendation system

This approach preprocesses the job description, generates embeddings, and computes cosine similarity between the job description and each resume. The similarity scores indicate how closely each resume matches the job description based on their embeddings.

**Setup and Installation**
Prerequisites
- python
- pdf2image
- docx2txt
- PyPDF2
- json
- numpy
- nltk



In [ ]:
!pip install langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinu

In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install pdf2image docx2txt PyPDF2

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=b67bb1d55fd16e4d1b2090c790ca851b7b4aa5f3ec3e560e14d8921c3c046445
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [ ]:
import os
import json
import requests
import docx2txt
import PyPDF2
import numpy as np

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
# importing all necessary modules
from gensim.models import Word2Vec, KeyedVectors
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

warnings.filterwarnings(action='ignore')

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def dim(a):
    if not type(a) == list:
        return []
    return [len(a)] + dim(a[0])

This function extracts text from a PDF file using the PyPDF2 library.



In [ ]:
def extract_text_from_pdf_file(uploaded_file):
    # Use PdfReader to read the text content from a PDF file
    pdf_reader = PyPDF2.PdfReader(uploaded_file)
    text_content = ""
    for page in pdf_reader.pages:
        text_content += str(page.extract_text())
    return text_content



This function extracts text from a DOCX file using the docx2txt library.

In [ ]:
def extract_text_from_docx_file(uploaded_file):
    # Use docx2txt to extract text from a DOCX file
    return docx2txt.process(uploaded_file)

This code snippet is used to extract text from a list of PDF resumes and store the extracted text in a list.

List of Resume Files:
**resume_list** contains the filenames of the PDF resumes you want to process.

**all_resume_text** is an empty list that will be used to store the text extracted from each resume.

In [ ]:
resume_list=["Vihan_IIT Kanpur_Resume.pdf.pdf","210867_riya_silotiya_bt_ee.pdf"]

all_resume_text=[]
for resume in resume_list:
    resume_text = extract_text_from_pdf_file(resume)
    all_resume_text.append(resume_text)
print(len(all_resume_text))

2


The code iterates over each filename in resume_list. For each file, it calls extract_text_from_pdf_file to extract the text content and appends it to all_resume_text.



---



# **Job description**

In [ ]:
job_description = '''About the job
EdgeVerve is a global leader in AI, Automation, and Analytics. It is a wholly-owned subsidiary of Infosys. EdgeVerve develops innovative software products and offers them on-premise or as cloud-hosted business platforms. Its technology empowers enterprises globally to bring life to their digital transformation initiatives. EdgeVerve aims to create a world where its technology augments human intelligence and creates possibilities for enterprises to thrive

 Product :

EDGE Platform, part of Infosys Topaz, bridges silos in people, processes, data, and technology for enterprises, amplifying the value of their existing digital core investments.
EDGE Platform platform acts as a transformative overlay, seamlessly integrating with enterprises' existing systems. It enables businesses to adopt cutting-edge capabilities, enhance operational efficiency, and unlock new opportunities for growth and innovation, all without disrupting their foundational systems.

Role : Product Manager

Responsibilities :

 - Partner with Internal and External stakeholders and contribute in building overall product strategy
 - Helps to define the product vision
 - Communicate with all stakeholders, prepare project plans, release notes, etc., and conduct timely business-technology governance meetings
 - Work with engineering /UX teams to provide comprehensive business solutions
 - B.tech/MBA


 Experience Required :

 - 9+ years in the industry with minimum of 5+ years in product Management in
 - Prior Experience in AI enabled / using products - NLP or Computer vision or ML or AI
 - Familiarity with various generative AI models, such as generative adversarial networks (GANs), variational autoencoders (VAEs), and transformer models, is important to understand their strengths, limitations, and use cases.
 - Experience in transformer models like BERT, ContractBERT
 - Machine Learning, AI, Language models experience, or experience in AI builder products.
 - Limited experience in Data science and data engineering is a plus
 - Scaled agile product management principles
Qualifications'''

In [ ]:
# messages=all_resume_text[1]
# corpus = []
# review = re.sub('[^a-zA-Z]', ' ', messages)
# review = review.lower()
# review = review.split()

# review = [lemmatizer.lemmatize(word) for word in review]
# review = ' '.join(review)
# corpus.append(review)

# Data Preprocessing


The preprocess function is designed to prepare text for further analysis by:

- Converting the text to lowercase

- Removing non-alphabetic characters,
Tokenizing the text
- Removing common stop words,
- Reassembling the cleaned words into a single string.

In [ ]:
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

- stop_words is a set of common English stop words (like "and", "the", "is") that will be removed from the text.
- The text is converted to lowercase to ensure that the text processing is case-insensitive

- re.sub is used to remove any characters that are not letters or whitespace. This step cleans up the text by removing punctuation and numbers.





This code preprocesses the text extracted from each resume using the preprocess function and stores the preprocessed text in a list.

In [ ]:
# Preprocess the text from all resumes
preprocessed_resumes = [preprocess(resume) for resume in all_resume_text]

In [ ]:
from google.colab import userdata
HF_TOKEN =userdata.get('HF_TOKEN')

In [ ]:
# @title
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model= "sentence-transformers/all-MiniLM-L6-v2",
    task="feature-extraction",
    huggingfacehub_api_token=HF_TOKEN,
)
texts = ["Hello, world!", "How are you?"]
x=hf_embeddings.embed_documents(texts)


The HuggingFaceEndpointEmbeddings class is initialized with the specified model and task. The HF_TOKEN is used for authentication with the Hugging Face API.

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model= "mixedbread-ai/mxbai-embed-large-v1",
    task="feature-extraction",
    huggingfacehub_api_token=HF_TOKEN,
)
# texts = ["Hello, world!", "How are you?"]
# p=hf_embeddings.embed_documents(texts)


huggingfacehub_api_token: Hugging Face API token for authentication.



---



**Get Embeddings for Preprocessed Resumes:**

The embed_documents method of the hf_embeddings object is used to obtain embeddings for the preprocessed resume texts.

In [ ]:

# Get embeddings using hf_embeddings object
embeddings = hf_embeddings.embed_documents(preprocessed_resumes)

In [ ]:
dim(embeddings )

[2, 1024]

In [ ]:
preprocessed_resumes[0]

'vihan kochatta senior undergraduate nednvihan kochatta gtbvihankochatta department chemistry bvihankiitkacin academic qualifications year degreecertificate institute cpi present bachelor science indian institute technology kanpur higher secondary mp board milestone academy jaora higher school cbse saint pauls convent school jaora achievements accomplished star coder rating codechef rating atcodeforces star rating badge hackerrank ranked th icpc asia amritapuri qualifier underscoring proficiency advanced problemsolving skills received citation certificate madhya pradesh government pratibhashali chhatra protsahan yojana internship experience software engineer deepinsights harvard business school oct nov objective engineered program acquire clients zooms access token facilitating access call recordings storage approachenabled email google authentication via django oauth postgresql serving users devised zoom auth reducing login time utilized zoom apis client access token retrieval impleme

Preprocess the Job Description:

In [ ]:
preprocessed_jobdescription=preprocess(job_description)

In [ ]:
texts = [preprocessed_resumes[0],preprocessed_jobdescription]
p=hf_embeddings.embed_documents(texts)


In [ ]:
texts = [preprocessed_resumes[0], preprocessed_resumes[1]]
hf_embeddings.embed_documents(texts)


[[-0.007117031142115593,
  -0.11288578063249588,
  -0.1700880527496338,
  -0.42818403244018555,
  -0.03862742334604263,
  0.5020278692245483,
  -0.22473298013210297,
  -0.6065346598625183,
  0.3554212749004364,
  0.7085434794425964,
  -0.11842004209756851,
  0.37654075026512146,
  0.5196467638015747,
  -0.8613457679748535,
  -0.42221716046333313,
  -0.009416583925485611,
  -0.4524606168270111,
  -0.230352982878685,
  -0.4833175837993622,
  0.1460975855588913,
  -0.0016402191249653697,
  0.6915345788002014,
  -1.2074670791625977,
  -0.36827749013900757,
  -0.2008030116558075,
  0.7616128921508789,
  -0.14026480913162231,
  0.3923168480396271,
  1.3438807725906372,
  0.48023277521133423,
  -0.152918741106987,
  0.06280378252267838,
  -0.0035918401554226875,
  -0.8998633623123169,
  -0.39768627285957336,
  0.2029799073934555,
  0.2547838091850281,
  -0.4727652370929718,
  -0.5560646653175354,
  -0.09715631604194641,
  0.05623659864068031,
  -0.6082016825675964,
  0.3762907385826111,
  -0.

In [ ]:
dim(p)

[2, 1024]

In [ ]:
preprocessed_resumes[1]

'riya silotiya riyasiitkacin rd year undergraduate riya silotiya bachelor technologybtech riyasilotiya gtb department electrical engineering ne academic qualification year degreecertificate institute cpi present btech indian institute technology kanpur cbse aisscexii railway senior secondary school bandikui cbse aissex bandikui public school bandikui work experience battery health modelling datadriven approach surge intern profswathi battula gtb mayjuly objective predict state health soh battery using recurrent neural networks rnns approachextracted data matlab files containing battery performance data nasa data repository performed feature selection anddata visualization identify crucial features predicting soh developed parallel rnnarchitecture using three different rnns vanilla rnn simplernn long shortterm memory lstm gated recurrent unit gru incorporating input features two distinct ways applying moving average filter data smoothing trained model using tensorflow keras l regulariza

The code calculates the cosine similarity between two sets of text embeddings.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Assuming hf_embeddings.embed_documents gives a list of embeddings
embedding_1 = hf_embeddings.embed_documents([preprocessed_resumes[0]])  # embedding for first document
embedding_2 = hf_embeddings.embed_documents([preprocessed_jobdescription])  # embedding for second document

# Convert to numpy arrays and reshape to 2D
embedding_1 = np.array(embedding_1[0]).reshape(1, -1)
embedding_2 = np.array(embedding_2[0]).reshape(1, -1)

# Calculate cosine similarity
similarity_score = cosine_similarity(embedding_1, embedding_2)[0][0]

print(f"Similarity score: {similarity_score}")


Similarity score: 0.6069767848369303


hf_embeddings.embed_documents is called to get embeddings for the first document (a preprocessed resume) and the second document (a preprocessed job description). These embeddings are lists of vectors

The embeddings are converted from lists to NumPy arrays and reshaped to be 2D arrays. This is required because cosine_similarity expects 2D arrays as input, where each row represents a vector.

cosine_similarity computes the cosine similarity between the two vectors.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Assuming hf_embeddings.embed_documents gives a list of embeddings
embedding_1 = hf_embeddings.embed_documents([preprocessed_resumes[1]])  # embedding for first document
embedding_2 = hf_embeddings.embed_documents([preprocessed_jobdescription])  # embedding for second document

# Convert to numpy arrays and reshape to 2D
embedding_1 = np.array(embedding_1[0]).reshape(1, -1)
embedding_2 = np.array(embedding_2[0]).reshape(1, -1)

# Calculate cosine similarity
similarity_score = cosine_similarity(embedding_1, embedding_2)[0][0]

print(f"Similarity score: {similarity_score}")


Similarity score: 0.5688326060950651


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Assuming hf_embeddings.embed_documents gives a list of embeddings
embedding_1 = hf_embeddings.embed_documents(["texts are"])  # embedding for first document
embedding_2 = hf_embeddings.embed_documents(["another text"])  # embedding for second document

# Convert to numpy arrays and reshape to 2D
embedding_1 = np.array(embedding_1[0]).reshape(1, -1)
embedding_2 = np.array(embedding_2[0]).reshape(1, -1)

# Calculate cosine similarity
similarity_score = cosine_similarity(embedding_1, embedding_2)[0][0]

print(f"Similarity score: {similarity_score}")


Similarity score: 0.8361307301271471
